In [1]:
from azureml.core import Workspace, Experiment, Environment

ws = Workspace.from_config()#get(name="udacity-project")#, resource_group='aml-vm-174174')

# myenv = Environment.get(workspace=ws, name="AzureML-Minimal")
# ws.get_details()

exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-175118
Azure region: southcentralus
Subscription id: 9b72f9e6-56c5-4c16-991b-19c652994860
Resource group: aml-quickstarts-175118


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
compute_cluster_name = "cpucluster" #was cpu_cluster_name='aml-compute'

# Verify that cluster does not exist already
try:
    compute_cluster = ComputeTarget(workspace=ws, name=compute_cluster_name) #was cpu_cluster #!!!target
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # To use a different region for the compute, add a location='<region>' parameter
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute_cluster = ComputeTarget.create(ws, compute_cluster_name, compute_config)

compute_cluster.wait_for_completion(show_output=True)
# print(compute_target.get_status().serialize())


InProgress.
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
# compute_targets = ws.compute_targets
# for compute_name, compute_type in compute_targets.items():
#     print(compute_name, compute_type.type, compute_type.provisioning_state)

In [4]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

# Specify parameter sampler
# experiment=Experiment(ws,'quick-starts-ws-175073')

ps = RandomParameterSampling( {
       "--C": choice (0.01,0.1,1),
       "--max_iter": choice(16,32,64,128)#choice(20,40,60,100,150,200)#choice(16,32,64,128) 
   })
# ({"learning_rate":uniform(0.05,0.1),"batch_size":uniform(16,32,64,128)})### YOUR CODE HERE ###

# Specify a Policy
policy = BanditPolicy(slack_factor=0.1,evaluation_interval=2)### YOUR CODE HERE ###
#slack_factor=0.15, evaluation_interval=1, delay_evaluation=5
if "training" not in os.listdir():
    os.mkdir("./training")

# import shutil
# shutil.copy('./train.py', './training')
script_folder='./training'
os.makedirs(script_folder,exist_ok=True)
import shutil
shutil.copy('./train.py',script_folder)
est=SKLearn( source_directory=script_folder,compute_target=compute_cluster,entry_script="train.py")

#Setup Environment for your training run
# from azureml.core.environment import Environment

# sklearn_run=Environment.from_conda_specification(name='sklearn-env',file_path='conda_dependencies.yml')

#Create a ScriptRunConfig Object to specify the configuration details of your trianing job
# from azureml.core import ScriptRunConfig
# est=ScriptRunConfig(source_directory="./training", script='train.py',compute_target=compute_target)

# Create a SKLearn estimator for use with train.py
# est = SKLearn(source_directory='./training', entry_script='train.py', compute_target=compute_target,vm_size='STANDARD_D2_V2')### YOUR CODE HERE ###
#train.py should be inside ./training folder! ##!!!!cpu_cluster_name
# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.

hyperdrive_config = HyperDriveConfig(hyperparameter_sampling=ps, primary_metric_name='Accuracy',primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,max_total_runs=20,policy=policy, estimator=est) #, estimator=est
#run_config=est
### YOUR CODE HERE ### #primary_metric_name="AUC_weighted" results in error!! Accuracy
##accuracy can be used as well

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.
'enabled' is deprecated. Please use the azureml.core.runconfig.DockerConfiguration object with the 'use_docker' param instead.


In [5]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hyperdrive_run=exp.submit(config=hyperdrive_config) #was pipeline_run
RunDetails(hyperdrive_run).show()
# pipeline_run.get_status()
hyperdrive_run.wait_for_completion(show_output=True) #show_output=True don't include otherwise error

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_4ac541c7-2cef-4dc2-9386-66f7f729eb1f
Web View: https://ml.azure.com/runs/HD_4ac541c7-2cef-4dc2-9386-66f7f729eb1f?wsid=/subscriptions/9b72f9e6-56c5-4c16-991b-19c652994860/resourcegroups/aml-quickstarts-175118/workspaces/quick-starts-ws-175118&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

"<START>[2022-01-13T17:42:59.602720][API][INFO]Experiment created<END>\n""<START>[2022-01-13T17:43:00.479357][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2022-01-13T17:43:01.264829][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_4ac541c7-2cef-4dc2-9386-66f7f729eb1f
Web View: https://ml.azure.com/runs/HD_4ac541c7-2cef-4dc2-9386-66f7f729eb1f?wsid=/subscriptions/9b72f9e6-56c5-4c16-991b-19c652994860/resourcegroups/aml-quickstarts-175118/workspaces/quick-starts-ws-175118&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254



{'runId': 'HD_4ac541c7-2cef-4dc2-9386-66f7f729eb1f',
 'target': 'cpucluster',
 'status': 'Completed',
 'startTimeUtc': '2022-01-13T17:42:59.319712Z',
 'endTimeUtc': '2022-01-13T17:51:05.907271Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name": "Accuracy", "goal": "maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '16accf39-d63b-4fcd-92cf-679e418ad5ee',
  'user_agent': 'python/3.6.9 (Linux-5.4.0-1063-azure-x86_64-with-debian-buster-sid) msrest/0.6.21 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.36.0',
  'space_size': '12',
  'score': '0.9074355083459787',
  'best_child_run_id': 'HD_4ac541c7-2cef-4dc2-9386-66f7f729eb1f_2',
  'best_metric_status': 'Succeeded'},
 'inputDatasets': [],
 'outputDatasets': [],
 'logFiles': {'azureml-logs/hyperdrive.txt': 'https://mlstrg175118.blob.core.windows.net/azureml/ExperimentRun/dcid.HD_4ac541c7-2cef-4dc2-9386-66f7f729eb1f/azureml-lo

In [6]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run=hyperdrive_run.get_best_run_by_primary_metric()
# gives error-suggests that the best_run has the value equivalence of null in other languages, which means it des not contain 
#anything. This means that the there was no result from pipeline_run.get_best_run_by_primary_metric() as used in the notebook
#to retrieve the best run, and thus nothing was assigned the best_run object. The question then would be: did the
#training complete successfully?

# print('fiel names {}'.format(best_run.get_file_names()))
print("best run file names :", best_run.get_file_names())


model = best_run.register_model(model_name = 'best_model', model_path = 'outputs/model.joblib')
#, tags={"Method":"HyperDrive"}, properties={"Accuracy":best_run.get_metrics()["Accuracy"]})
#https://github.com/Azure/MachineLearningNotebooks/issues/269
#!!!!

best run file names : ['logs/azureml/15_azureml.log', 'outputs/model.joblib', 'system_logs/cs_capability/cs-capability.log', 'system_logs/hosttools_capability/hosttools-capability.log', 'system_logs/lifecycler/execution-wrapper.log', 'system_logs/lifecycler/lifecycler.log', 'system_logs/lifecycler/vm-bootstrapper.log', 'user_logs/std_log.txt']


In [7]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
url="https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds=TabularDatasetFactory.from_delimited_files(path=url)

In [8]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)### YOUR DATA OBJECT HERE ###)

In [10]:
from azureml.train.automl import AutoMLConfig
from sklearn.model_selection import train_test_split

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.

import pandas as pd
# x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)
x_all=pd.concat([x,y],axis=1) #training_data below should also have the label column

automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric="accuracy", #AUC_weighted
    training_data=x_all,#ds,

    label_column_name='y',
    n_cross_validations=5,
    
    enable_onnx_compatible_models=True)

In [11]:
# Submit your automl run
run=exp.submit(config=automl_config,show_output=True)
### YOUR CODE HERE ###

No run_configuration provided, running on local with default configuration
Running in the active local environment.


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_2f1bd284-7348-4e06-b2e5-637465a1e2c3,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias toward

INFO:interpret_community.common.explanation_utils:Using default datastore for uploads


In [12]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
run.get_metrics()

import onnxmltools

best_run2, fitted_model = run.get_output(return_onnx_model=True)
#https://docs.microsoft.com/en-us/azure/machine-learning/tutorial-auto-train-models

onnxmltools.utils.save_model(fitted_model, './outputs/best_automl.onnx')

In [ ]:
compute_cluster.delete() #cpu_cluster
#https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.compute.computetarget?view=azure-ml-py#delete--